In [1]:
import pygame
import numpy as np
#from stable_baseline3.stable_baselines3.ppo import ppo
from tensorflow import keras
import tensorflow as tf
from tensorflow import concat
import time
#import tensorflow_probability as tfp
import tensorflow.keras.losses as kls

In [2]:
class MowerEnv2():
    def __init__(self, info, render_mode = True): #info 0 = len, 1 = number of agents (up to 4)
        self.render_mode = render_mode
        self.len = info[0]
        self.agents = info[1]
        if render_mode == True:
            pygame.init()
            self.screen = pygame.display.set_mode((720, 720))
        # Actions we can up down left right
        # Set length
        self.direct = [[-1, 0], [0, 1], [1, 0], [0, -1]]

    def step(self, action):
        ret = []
        for j in range(self.agents):
            # Apply action (no walls for now)
            x = self.all[j][0] + self.direct[action[j]][0]
            y = self.all[j][1] + self.direct[action[j]][1]
            if self.isValid(x, y) == 0:
                x = self.all[j][0]
                y = self.all[j][1]
            if self.state[x][y][0] == 1:
                self.area_clear += 1
                reward = 5.0 * np.power(4, self.area_clear/self.max_area)
            else:
                reward = -0.5
                self.running_length -= 1
            self.state[x][y][0] = 0
            self.all[j][0] = x
            self.all[j][1] = y

            # update boundries
            for i in range(4):
                self.all[j][i+2] = self.isValid(x + self.direct[i][0], y + self.direct[i][1])
            temp = self.cheats(x, y)
            self.all[j][6]= self.all[j][7]= self.all[j][8]= self.all[j][9] = 0
            for p in temp:
                self.all[j][6+p] = 1
            # Check if  done
            if self.running_length < 0:
                done = True
            else:
                if self.finished(): #change back to self.finished()
                    if not done:
                        reward = self.max_area * 20
                    else:
                        reward = 0
                    done = True
                else:
                    done = False
            ret.append(reward)
        # Set placeholder for info
        info = {}

        # Return step information
        return (ret, done)

    def isValid(self, x, y):
        if x < 0 or x < 0 or x >= self.len or y < 0 or y >= self.len:
            return 0 #return False
        if self.state[x][y][1] == 1:
            return 0
        return 1 #return true

    def cheats(self, posx, posy):
        # Next action:
        # (feed the observation to your agent here)
        calc = np.zeros((16, 16))
        list = deque()
        calc[posx][posy] = 1
        list.append([posx, posy])
        move = []
        while True: #has multiple paths issues, but should be irrelevent
            size = len(list)
            for z in range(size):
                tuple = list.popleft()
                for i in range(4):
                    x = tuple[0] + self.direct[i][0]
                    y = tuple[1] + self.direct[i][1]
                    if x >= 0 and x < 16 and y >=0 and y < 16:
                        if self.state[x][y][1] == 0 and calc[x][y] == 0:
                            calc[x][y] = 1
                            choice = i
                            if len(tuple) == 3:
                                choice = tuple[2]
                            if self.state[x][y][0] == 0:
                                ret = [x, y, choice]
                                list.append(ret)
                            else:
                                move.append(choice)
                                break

            if len(move) > 0:
                break
        return move

    def finished(self):
        return self.area_clear==self.max_area

    def render(self):
        # Implement viz
        if self.render_mode == True:
            pygame.event.get()
            gap = 720//self.len
            for i in range(0, 720, gap):
                for j in range(0, 720, gap):
                    if self.state[i//gap][j//gap][0] == 0:
                        color = (255, 255, 255)
                    elif self.state[i//gap][j//gap][0] == 1:
                        color = (255, 255,0)
                    for k in range(self.agents):
                        if self.all[k][0] == i//gap and self.all[k][1] == j//gap:
                            color = (255, 0, 0)
                    if self.state[i//gap][j//gap][1] == 1:
                        color = (0, 0, 0)
                    pygame.draw.rect(self.screen, color, (j, i, gap, gap))
            pygame.display.update()


    def reset(self, seed=None, options=None):
        # Set starting state
        self.state = np.copy(combined_data)
        self.all = np.zeros((self.agents, 10), dtype = int)

        for i in range(self.agents):# Set position
            all[i][0] = corners[i][0]
            all[i][1] = corners[i][1]

            # Set border detect 0 = blocked, 1 = valid
            for j in range(4):
                all[i][j+2] = self.isValid(all[i][0] + self.direct[j][0], all[i][1] + self.direct[j][1])
            temp = self.cheats(all[i][0], all[i][1])
            all[i][6]= all[i][7]= all[i][8]= all[i][9] = 0
            for p in temp:
                all[i][6+p] = 1

        self.running_length = ((self.len) * (self.len) * (1 + self.agents)) // 2
        self.area_clear = 0
        self.max_area = np.sum(self.state['grid'][0])
        return self.state, {}

    def get_state(self):
        return (self.state, self.all)

    def close(self):
        pass

grass_data = [[0,0,1,0,1,0,1,0,0,1,1,0,1,0,0,0],
[0,0,1,0,1,0,1,1,0,1,0,0,1,0,0,1],
[1,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0],
[0,0,0,1,0,1,0,1,0,0,0,1,1,0,1,0],
[0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0],
[1,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0],
[1,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0],
[0,1,1,0,0,0,1,0,0,0,1,0,1,0,0,0],
[0,0,0,1,1,1,0,0,0,0,1,0,1,0,0,0],
[1,0,0,1,0,1,0,1,0,1,1,0,1,0,1,0],
[1,0,0,0,1,1,1,0,1,0,0,0,0,0,1,0],
[0,1,0,0,1,0,1,1,1,0,1,1,0,0,0,0],
[1,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0],
[1,0,0,1,0,0,1,0,1,0,1,1,1,1,0,0],
[1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0],
[1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
walls_data = [[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0],
[0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0],
[0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0],
[0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1],
[0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0],
[0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0],
[0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0],
[0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0],
[0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0],
[0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0],
[0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0],
[0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0],
[0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]]
corners = [[0, 0], [1, 1], [1, 0], [0, 1]]
walls_data = [] #delete later
for i in range(16):
    walls_data.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
corners = [[0, 0], [1, 1], [1, 0], [0, 1]]
combined_data = []
for i in range(16):
    combined_data.append([])
    for j in range(16):
        combined_data[i].append([grass_data[i][j], walls_data[i][j]])
combined_data = np.array(combined_data, dtype = int)
#@title


In [3]:
class critic(keras.Model):
    def __init__(self):
        super().__init__()
        self.m1 = keras.Sequential([
            keras.layers.Input((16, 16, 2)), #change later
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last"),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last"),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            keras.layers.Flatten(data_format="channels_last"),
            keras.layers.Dense(units=64, activation="relu")
        ])
        self.m2 = keras.Sequential([
            keras.layers.Input((10,)),
            keras.layers.Dense(units=64, activation = 'relu')
        ])
        self.m3 = keras.Sequential([
            keras.layers.Input((128,)),
            keras.layers.Dense(units=64, activation="relu"),
            keras.layers.Dense(units=64, activation="relu"),
            keras.layers.Dense(units=1, activation="linear")
        ])

    def call(self, input_image, input_data):
        I = self.m1(input_image)
        D = self.m2(input_data)
        O = self.m3(concat([I,D],1))
        return O
    

class actor(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.m1 = keras.Sequential([
            keras.layers.Input((16, 16, 2)), #change later
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last"),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last"),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            keras.layers.Flatten(data_format="channels_last"),
            keras.layers.Dense(units=64, activation="relu")
        ])
        self.m2 = keras.Sequential([
            keras.layers.Input((10,)),
            keras.layers.Dense(units=64, activation = 'relu')
        ])
        self.m3 = keras.Sequential([
            keras.layers.Input((128,)),
            keras.layers.Dense(units=64, activation="relu"),
            keras.layers.Dense(units=64, activation="relu"),
            keras.layers.Dense(units=4, activation="softmax")
        ])

    def call(self, input_image, input_data):
        I = self.m1(input_image)
        D = self.m2(input_data)
        O = self.m3(concat([I,D],1))
        return O
"""
class critic(keras.Model):
    def __init__(self):
        super().__init__()
        self.m1 = keras.Sequential([
            keras.layers.Input((16, 16, 2)), #change later
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last", kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            tf.keras.layers.Dropout(.2),
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last", kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            tf.keras.layers.Dropout(.2),
            keras.layers.Flatten(data_format="channels_last"),
            keras.layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.L2(0.01))
        ])
        self.m2 = keras.Sequential([
            keras.layers.Input((10,)),
            keras.layers.Dense(units=64, activation = 'relu', kernel_regularizer=keras.regularizers.L2(0.01))
        ])
        self.m3 = keras.Sequential([
            keras.layers.Input((128,)),
            keras.layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.L2(0.01)),
            tf.keras.layers.Dropout(.2),
            keras.layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.L2(0.01)),
            tf.keras.layers.Dropout(.2),
            keras.layers.Dense(units=1, activation="linear", kernel_regularizer=keras.regularizers.L2(0.01))
        ])

    def call(self, input_image, input_data):
        I = self.m1(input_image)
        D = self.m2(input_data)
        O = self.m3(concat([I,D],1))
        return O
    

class actor(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.m1 = keras.Sequential([
            keras.layers.Input((16, 16, 2)), #change later
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last", kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            tf.keras.layers.Dropout(.2),
            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),
            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),
activation='relu',data_format="channels_last", kernel_regularizer=keras.regularizers.L2(0.01)),
            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),
            tf.keras.layers.Dropout(.2),
            keras.layers.Flatten(data_format="channels_last"),
            keras.layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.L2(0.01))
        ])
        self.m2 = keras.Sequential([
            keras.layers.Input((10,)),
            keras.layers.Dense(units=64, activation = 'relu', kernel_regularizer=keras.regularizers.L2(0.01))
        ])
        self.m3 = keras.Sequential([
            keras.layers.Input((128,)),
            keras.layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.L2(0.01)),
            tf.keras.layers.Dropout(.2),
            keras.layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.L2(0.01)),
            tf.keras.layers.Dropout(.2),
            keras.layers.Dense(units=4, activation="softmax", kernel_regularizer=keras.regularizers.L2(0.01))
        ])

    def call(self, input_image, input_data):
        I = self.m1(input_image)
        D = self.m2(input_data)
        O = self.m3(concat([I,D],1))
        return O
"""

'\nclass critic(keras.Model):\n    def __init__(self):\n        super().__init__()\n        self.m1 = keras.Sequential([\n            keras.layers.Input((16, 16, 2)), #change later\n            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),\n            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),\nactivation=\'relu\',data_format="channels_last", kernel_regularizer=keras.regularizers.L2(0.01)),\n            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),\n            tf.keras.layers.Dropout(.2),\n            keras.layers.ZeroPadding2D(padding = (1, 1), data_format="channels_last"),\n            keras.layers.Conv2D(filters = 16,kernel_size = (3,3),strides = (1,1),\nactivation=\'relu\',data_format="channels_last", kernel_regularizer=keras.regularizers.L2(0.01)),\n            keras.layers.MaxPooling2D(pool_size=(2, 2), data_format="channels_last"),\n            tf.keras.layers.Dropout(.2),\n            keras.la

In [21]:
class agent():
    def __init__(self):
        self.a_opt = keras.optimizers.Adam(learning_rate=0.0002) # or 0.00015
        self.c_opt = keras.optimizers.Adam(learning_rate=0.0002)
        self.actor = actor()
        self.critic = critic()
        self.clip_pram = 0.2

    def act(self, stateI, stateN):
        prob = self.actor(np.expand_dims(stateI, 0), np.expand_dims(stateN, 0))
        prob = prob.numpy()
        #dist = tfp.distributions.Categorical(probs=prob, dtype=tf.float32)
        #action = dist.sample()
        #return int(action.numpy()[0])
        return np.random.choice(a = [0, 1, 2, 3], size = 1, p = prob.flatten())
    
    def learn(self, statesI, statesN, actions,  adv , old_probs, discnt_rewards):
        discnt_rewards = tf.reshape(discnt_rewards, (len(discnt_rewards),))
        adv = tf.reshape(adv, (len(adv),))

        #old_p = old_probs

        #old_p = tf.reshape(old_p, (len(old_p),2))
        with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
            p = self.actor(tf.stack(statesI, 0),tf.stack(statesN, 0), training=True)
            v =  self.critic(np.stack(statesI, 0),np.stack(statesN, 0),training=True)
            #v = tf.reshape(v, (len(v),))
            #td = tf.math.subtract(discnt_rewards, v)
            c_loss = 0.5 * kls.mean_squared_error(np.expand_dims(discnt_rewards, 0), v)
            a_loss = self.actor_loss(p, actions, adv, old_probs, c_loss)
            
        grads1 = tape1.gradient(a_loss, self.actor.trainable_variables)
        grads2 = tape2.gradient(c_loss, self.critic.trainable_variables)
        self.a_opt.apply_gradients(zip(grads1, self.actor.trainable_variables))
        self.c_opt.apply_gradients(zip(grads2, self.critic.trainable_variables))
        return a_loss, c_loss
    def actor_loss(self, probs, actions, adv, old_probs, closs):
        
        probability = probs      
        entropy = tf.reduce_mean(tf.math.negative(tf.math.multiply(probability,tf.math.log(probability))))
        #print(probability)
        #print(entropy)
        sur1 = []
        sur2 = []
        
        for pb, t, op, a in zip(probability, adv, old_probs, actions):
                        t =  tf.constant(t)
                        #op =  tf.constant(op)
                        #print(f"t{t}")
                        #ratio = tf.math.exp(tf.math.log(pb + 1e-10) - tf.math.log(op + 1e-10))
                        ratio = tf.math.divide(pb[a],op[a])
                        #print(f"ratio{ratio}")
                        s1 = tf.math.multiply(ratio,t)
                        #print(f"s1{s1}")
                        s2 =  tf.math.multiply(tf.clip_by_value(ratio, 1.0 - self.clip_pram, 1.0 + self.clip_pram),t)
                        #print(f"s2{s2}")
                        sur1.append(s1)
                        sur2.append(s2)

        sr1 = tf.stack(sur1)
        sr2 = tf.stack(sur2)
        
        #closs = tf.reduce_mean(tf.math.square(td))
        loss = tf.math.negative(tf.reduce_mean(tf.math.minimum(sr1, sr2)) - closs + 0.0001 * entropy)
        #print(loss)
        return loss

In [6]:
#useless
"""
def test_reward(env):
    total_reward = 0
    state = env.reset()
    done = False
    while not done:
        action = np.argmax(agentoo7.actor(np.array([state])).numpy())
        next_state, reward, done, _ = env.step(action)
        state = next_state
        total_reward += reward

    return total_reward
"""

'\ndef test_reward(env):\n    total_reward = 0\n    state = env.reset()\n    done = False\n    while not done:\n        action = np.argmax(agentoo7.actor(np.array([state])).numpy())\n        next_state, reward, done, _ = env.step(action)\n        state = next_state\n        total_reward += reward\n\n    return total_reward\n'

In [5]:
def preprocess1(reward, done, value, gamma):
    g = 0
    lmbda = 0.95
    returns = []

    for i in reversed(range(len(reward))):
        delta = reward[i] + gamma * value[i + 1] * done[i] - value[i]
        g = delta + gamma * lmbda * done[i] * g
        returns.append(g + value[i])

    returns.reverse()
    adv = np.array(returns, dtype=np.float32) - value[:-1]
    adv = (adv - np.mean(adv)) / (np.std(adv) + 1e-10)
    returns = np.array(returns, dtype=np.float32)
    return returns, adv    

In [ ]:
copy  = [agent(),agent(),agent(),agent()]
for i in range(4):
    copy[i].critic.set_weights(agent_list[i].critic.get_weights())
    copy[i].actor.set_weights(agent_list[i].actor.get_weights())

In [ ]:
for i in range(4):
    copy[i].critic.set_weights(agent_list[i].critic.get_weights())
    copy[i].actor.set_weights(agent_list[i].actor.get_weights())

In [22]:
tf.random.set_seed(369)
agent_list = [agent(),agent(),agent(),agent()]

In [23]:
steps = 10000
target = False
number = 1
wid = 16
env = MowerEnv2([wid, number], False)
rew_avg = 0
for s in range(steps):
    #if target == True:
    #    break
    total_rew = 0
    done = False
    rewards = []
    statesI = []
    statesN = []
    actions = []
    probs = []
    dones = []
    values = []
    for j in range(number):
        rewards.append([])
        statesN.append([])
        actions.append([])
        probs.append([])
        values.append([])

    env.reset()
    i = 0
    while True:
        stateI, stateN = env.get_state()
        stateI = np.copy(stateI)
        stateN = np.copy(stateN)
        statesI.append(stateI)
        for j in range(number):
            action = agent_list[j].act(stateI, stateN[j])[0]
            actions[j].append(action)
            value = agent_list[j].critic(np.expand_dims(stateI,0), np.expand_dims(stateN[j],0)).numpy()[0]
            values[j].append(value)
            statesN[j].append(stateN[j])
            prob = agent_list[j].actor(np.expand_dims(stateI,0), np.expand_dims(stateN[j],0)).numpy()[0]
            probs[j].append(prob)

        _, reward, done, _, _ = env.step([actions[j][i] for j in range(number)])

        for j in range(number):
            rewards[j].append(reward[j])
            total_rew += reward[j]
            #actions.append(tf.one_hot(action, 2, dtype=tf.int32).numpy().tolist())

        dones.append(1-done)
        if done:
            break

        i += 1

    stateI, stateN = env.get_state()
    stateI = np.copy(stateI)
    stateN = np.copy(stateN)
    for j in range(number):
        value = agent_list[j].critic(np.expand_dims(stateI,0), np.expand_dims(stateN[j],0)).numpy()[0]
        values[j].append(value)
    #np.reshape(probs, (len(probs),2))
    #probs = np.stack(probs, axis=0)
    rets = []
    advs = []
    for j in range(number):
        returns, adv  = preprocess1(rewards[j], dones, values[j], 0.995)
        rets.append(returns)
        advs.append(adv)

    for epocs in range(5):
        for j in range(number):
            al,cl = agent_list[j].learn(statesI, statesN[j], actions[j], advs[j], probs[j], rets[j])
        # print(f"al{al}")
        # print(f"cl{cl}")

    rew_avg += total_rew
    if s % 4 == 3:
        print(f"{s}: {rew_avg/(s+1)}")


env.close()


3: -45.23266351076669
7: -37.4552417005677
11: -24.272375283951035
15: -21.939218665790303
19: -1.8959848746324284
23: 16.751326745405752
27: 33.83466812789782
31: 46.107850343037704
35: 48.02731903508372
39: 53.05615060831428
43: 63.25917083053732
47: 69.26065744626835
51: 71.7922668001573
55: 73.85904389501513
59: 81.61523524076046
63: 83.32098250549483
67: 94.17755336434811
71: 104.88856214333194
75: 113.16158512412309
79: 126.94137569668958
83: 135.80569999323137
87: 145.90781842481118
91: 160.56088579163546
95: 174.3958342112552
99: 187.8217439060194
103: 196.84858612146638
107: 205.35330786327663
111: 212.72605784888054
115: 223.8738393378396
119: 231.0652743092811
123: 239.78854556825482
127: 247.5824349180899
131: 257.79202903117687
135: 266.7704668037273
139: 273.84746035494055
143: 282.35475947624104
147: 287.02243963142627
151: 292.750025479691
155: 297.3019657452828
159: 303.4295913406553
163: 309.6349009724363
167: 315.7802086821695
171: 319.9003086757623
175: 325.90886591

KeyboardInterrupt: 

In [14]:
read = [-91.40332473078743
,
-50.36069344146
,
-24.976366240561006
,
-31.46056401381606
,
-18.397853638892826
,
-37.851799131618215
,
37.79446623850616
,
-56.48096115798328
,
1.9173916039322512
,
-31.46056401381606
,
-24.976366240561013
,
60.59755802567484
,
-4.952376633358192
,
144.15406580582882
,
45.28600162288956
,
144.1540658058288
,
144.15406580582882
,
1.9173916039322574
,
100.86821270734691
,
45.28600162288954
,
-37.851799131618215
,
-31.46056401381606
,
15.958150733042725
,
15.958150733042729
,
-18.397853638892826
,
-31.460564013816054
,
-44.15140438109258
,
-56.48096115798327
,
-56.48096115798327
,
-68.45951938699949
,
-68.45951938699949
,
-80.09707131545711
,
-50.36069344146
,
-85.79101448924057
,
-50.36069344146
,
-50.36069344146
,
-68.45951938699949
,
-50.36069344146
,
-56.48096115798327
,
-50.36069344146
,
-50.36069344146
,
8.887082921177253
,
317.05868406413435
,
8.88708292117725
,
60.59755802567484
,
210.2528266950486
,
23.13206959459773
,
45.28600162288957
,
-24.976366240561006
,
-50.36069344146
,
-37.851799131618215
,
-4.95237663335819
,
15.958150733042714
,
8.887082921177253
,
-37.85179913161821
,
-18.397853638892826
,
-4.952376633358185
,
-62.513483811985886
,
-74.32030783109575
,
-31.460564013816054
,
-44.151404381092576
,
-37.851799131618215
,
-50.36069344146
,
-4.952376633358199
,
-24.976366240561006
,
23.13206959459773
,
-31.46056401381606
,
-11.723654368042652
,
92.57980620350648
,
45.28600162288956
,
100.86821270734691
,
37.794466238506146
,
-31.46056401381606
,
23.132069594597723
,
15.958150733042714
,
-11.72365436804264
,
-31.460564013816057
,
68.42077201771373
,
-44.15140438109258
,
-68.45951938699949
,
-37.851799131618215
,
-4.952376633358183
,
-18.397853638892826
,
1.917391603932252
,
-56.48096115798327
,
-68.45951938699949
,
-31.460564013816054
,
-37.851799131618215
,
-50.36069344146
,
-50.36069344146
,
-44.15140438109258
,
-50.36069344146
,
-44.15140438109258
,
-31.460564013816068
,
-4.952376633358191
,
-18.39785363889282
,
-44.15140438109258
,
15.958150733042714
,
-37.851799131618215
,
-37.851799131618215
,
-18.397853638892812
,
-50.36069344146
,
-50.36069344146
,
-37.851799131618215
,
-31.460564013816054
,
-56.48096115798327
,
-44.15140438109258
,
-37.851799131618215
,
-56.48096115798327
,
-44.151404381092576
,
-31.460564013816068
,
-18.397853638892826
,
92.57980620350651
,
8.88708292117725
,
68.42077201771372
,
23.13206959459773
,
92.57980620350648
,
126.46381782255895
,
117.80845221249749
,
-11.723654368042654
,
-31.46056401381606
,
109.27717685988407
,
52.88650389863514
,
-4.952376633358188
,
8.887082921177253
,
30.410335508808664
,
-11.723654368042654
,
8.887082921177239
,
68.42077201771372
,
171.666099363308
,
126.46381782255892
,
162.3626776217865
,
45.28600162288956
,
109.27717685988407
,
153.19263718973983
,
52.88650389863514
,
30.410335508808643
,
52.88650389863513
,
37.79446623850614
,
8.88708292117725
,
30.410335508808657
,
92.57980620350651
,
68.42077201771372
,
109.27717685988407
,
60.59755802567485
,
135.24507862442542
,
23.13206959459771
,
230.39828004442384
,
68.42077201771372
,
68.42077201771372
,
76.3577772775547
,
162.36267762178653
,
117.80845221249749
,
45.28600162288956
,
181.10484249012492
,
-4.952376633358189
,
126.46381782255892
,
84.41022893730097
,
84.41022893730099
,
23.13206959459773
,
30.410335508808664
,
109.27717685988407
,
126.4638178225589
,
15.958150733042729
,
52.88650389863514
,
8.88708292117725
,
92.57980620350648
,
84.41022893730097
,
162.36267762178653
,
109.27717685988407
,
37.794466238506146
,
-4.952376633358199
,
-11.723654368042652
,
-37.851799131618215
,
15.958150733042732
,
8.887082921177253
,
-4.952376633358188
,
23.132069594597716
,
30.410335508808686
,
23.13206959459773
,
60.59755802567484
,
153.19263718973983
,
84.41022893730097
,
84.41022893730097
,
60.59755802567483
,
-18.397853638892826
,
117.80845221249749
,
60.59755802567486
,
126.46381782255892
,
-11.723654368042654
,
45.28600162288954
,
30.410335508808664
,
68.4207720177137
,
126.46381782255892
,
100.86821270734691
,
240.69130241085875
,
-4.952376633358189
,
144.15406580582882
,
68.4207720177137
,
100.86821270734691
,
23.132069594597716
,
162.36267762178653
,
162.36267762178653
,
76.35777727755472
,
190.68087529711067
,
117.80845221249749
,
162.3626776217865
,
-18.397853638892826
,
100.86821270734691
,
162.36267762178653
,
100.86821270734691
,
190.68087529711067
,
117.80845221249749
,
76.3577772775547
,
68.42077201771372
,
15.958150733042714
,
52.886503898635155
,
37.79446623850616
,
84.41022893730099
,
15.958150733042729
,
23.132069594597716
,
37.794466238506146
,
-4.952376633358195
,
30.410335508808664
,
60.597558025674815
,
-37.851799131618215
,
45.28600162288957
,
8.887082921177253
,
-50.36069344146
,
-18.39785363889282
,
-24.97636624056102
,
-37.851799131618215
,
45.28600162288957
,
-11.723654368042652
,
37.794466238506146
,
-56.48096115798327
,
-4.952376633358199
,
92.57980620350648
,
1.917391603932245
,
-11.723654368042652
,
68.4207720177137
,
1.9173916039322592
,
52.88650389863511
,
52.88650389863513
,
-4.952376633358189
,
45.28600162288956
,
45.28600162288956
,
37.79446623850616
,
30.410335508808643
,
-24.976366240561013
,
8.887082921177239
,
30.410335508808664
,
109.27717685988407
,
60.59755802567484
,
68.42077201771373
,
-11.723654368042654
,
45.28600162288957
,
60.59755802567483
,
30.410335508808664
,
117.80845221249749
,
92.57980620350648
,
68.42077201771372
,
100.86821270734691
,
135.24507862442545
,
84.41022893730099
,
68.42077201771372
,
153.19263718973983
,
126.46381782255892
,
84.41022893730097
,
92.57980620350648
,
52.886503898635134
,
23.13206959459773
,
153.1926371897398
,
52.88650389863513
,
117.80845221249749
,
92.57980620350648
,
76.3577772775547
,
153.19263718973983
,
144.15406580582882
,
68.42077201771372
,
68.42077201771372
,
84.41022893730099
,
109.27717685988407
,
-4.952376633358192
,
144.1540658058288
,
52.88650389863515
,
109.27717685988407
,
84.41022893730097
,
153.19263718973983
,
92.57980620350648
,
100.86821270734694
,
109.27717685988407
,
126.46381782255895
,
171.666099363308
,
135.24507862442542
,
45.28600162288956
,
109.27717685988407
,
52.886503898635134
,
92.57980620350651
,
30.41033550880865]

In [16]:
list = []
for i in range(300):
    list.append(sum(read[:(i+1)])/(i+1))
for i in range(9, 300, 10):
    print(list[i])

-30.268026852649648
17.690417579990196
1.3660342061229178
-14.252763169007313
-0.4341394807814767
-4.486038353509953
-4.480589742850745
-3.1840414609013967
-6.766468724948729
-9.063533336463841
-12.126376977485466
-7.21343110473829
-3.5639482622546734
2.258099496353668
8.020243669359838
13.121376689295184
17.011371927375375
16.509817752656485
19.419598561303445
23.051774213176813
27.38216707891772
29.597266711933973
28.72149333779043
27.649218006087615
27.763086219996477
28.695166051285405
31.360498274056738
33.578377711727754
35.502075790897266
37.565207296104845
